# Вступление
__Талантливый школьник во время сезона 2024 нарисовал новый логотип Красноярской Летней Школы. Оно так всем понравилось, что дирекция решила отсканировать это изображение и сохранить его на компьюетере. Получился файл формата BMP с расширением ".bmp"__

Эту картинку можно найти по ссылке (?):   

<img src="logo.bmp" width="200" height="200">

Известно, что байтовое представление .bmp изображения имеет следующую структуру :
1. BITMAPFILEHEADER - первые 14 байт
1. BITMAPINFO - в наших задачах это всегда будут следующие 40 байт
1. Пиксельные данные - всё остальное

Подробную информацию про формат можно прочитать на [википедии](https://ru.wikipedia.org/wiki/BMP#Таблица_цветов), но из всех этих данных нас будут интересовать в основном следующее:
1. с 0A по 0С (10-14) байт запсиано положение пиксельных данных относительно начала файла
1. с 12 по 16 (18-22) байт записана ширина изображения в пикселях
1. с 16 по 2A (22-26) байт записана высота изображения в пикселях. Если это число положительное, то строки пикселеей записаны снизу вверх. Если это число отрицательное, то запись строк идёт сверху вниз, причём количество строк равно абсолютному значению этого числа
1. с 1С по 1E (28-30) байт записано количество бит, которым шифруется каждый пиксель. В нашем случае - 24 бита = 3 байта

Пиксельные данные записаны последовательностью цветов пикселей по следующим правилам:
- Всё изображение разбито на однопиксельные строки, которые идут по порядку
- Каждый пиксель зашифрован тремя числами от 0 до 255 в палитре rgb - то есть значениями, красной, зелёной и синей компонент цвета пикселя. 
- Пиксели в каждой строке записаны строго слева направо. 
- В конце каждой строки стоит один байт "00" - конец строки.
- Порядок строк зависит от значения высоты, записанной в BITMAPINFO. Отрицательный - серху внизу, положительный - снизу вверх.

In [34]:
#Создадим пример
import PIL.Image
example = PIL.Image.new(mode='RGB', size=(2,2))
example.putpixel((0,0), (255, 0, 0))
example.putpixel((1,0), (0, 255, 0))
example.putpixel((0,1), (0, 0, 255))
example.putpixel((1,1), (255, 255, 255))
example.save('example.bmp', 'BMP')

<img src="example.bmp" width="200" height="200">

In [35]:
#С помощью википедии, поймём какая полезная информация зашита в заголовке
with open('example.bmp', 'rb') as logo:
    logo_bytes = logo.readline()
    print('BITMAPFILEHEADER содержит 14 байт - ', logo_bytes[0:14].hex())
    print('Размер файла в байтах : ', int.from_bytes(logo_bytes[2:6], byteorder='little'))
    print('Информация про пиксели начинается с позиции : ', int.from_bytes(logo_bytes[10:14], byteorder='little'))
    print('BITMAPINFO содержит байтов:', int.from_bytes(logo_bytes[14:18], byteorder='little'))
    print('Ширина в пикселях', int.from_bytes(logo_bytes[18:22], byteorder='little'))
    print('Высота в пикселях', int.from_bytes(logo_bytes[22:26], byteorder='little', signed=True)) # Отрицательное число - означает порядок строк сверху вниз
    print('Битность на пиксель:', int.from_bytes(logo_bytes[28:30], byteorder='little')) 
    print('Cпособ хранения данных:',int.from_bytes(logo_bytes[30:34], byteorder='little')) #0 - двумерный массив
    print('Размер пиксельных данных в байтах:',int.from_bytes(logo_bytes[34:38], byteorder='little'))

BITMAPFILEHEADER содержит 14 байт -  424d460000000000000036000000
Размер файла в байтах :  70
Информация про пиксели начинается с позиции :  54
BITMAPINFO содержит байтов: 40
Ширина в пикселях 2
Высота в пикселях 2
Битность на пиксель: 24
Cпособ хранения данных: 0
Размер пиксельных данных в байтах: 16


In [39]:
with open('example.bmp', 'rb') as logo:
    logo_bytes = logo.readline()
    print(logo_bytes[14:54].hex())
    print(logo_bytes[54:].hex())

280000000200000002000000010018000000000010000000c40e0000c40e00000000000000000000
ff0000ffffff00000000ff00ff000000


In [13]:
128*3/4

96.0

In [14]:
129/128

1.0078125

In [15]:
#Создадим пример
import PIL.Image
example = PIL.Image.new(mode='RGB', size=(4,1))
example.putpixel((0,0), (255, 0, 0))
example.putpixel((1,0), (235, 0, 0))
example.putpixel((2,0), (215, 0, 0))
example.putpixel((3,0), (195, 0, 0))
example.save('grad.bmp', 'BMP')

----

# Задание 1 (Варинат 1):   
Напишите функцию (метод) ```half_image(input_image, result_image)```
, которая заливает верхнюю половину BMP изображения чёрным цветом.    
У этой функции должно быть два аргумета:     
- ```input_image : string``` - относительный или абсолютный путь к BMP изображению описанного выше формата. Известно, что изображение имеет чётное колчиество пикселей по высоте.
- ```result_image : string``` - относительный или абсолютный путь, по которому будет записан результат выполнения функции.   
    
Эта функция (метод) должен использовать только функции стандартных библиотек вашего языка программирования.

In [2]:
# Возможное решение на python
def half_image(input_image : str, result_image: str):
    with open(input_image, 'rb') as inp:
        header = inp.read(54)
        width = int.from_bytes(header[18:22], byteorder='little')
        heigth = int.from_bytes(header[22:26], byteorder='little', signed=True)
        alignment = width * 3 % 4
        with open(result_image, 'wb') as result:
            result.writelines([header])
            
            for i in range(heigth):
                if i >= heigth/2: 
                    result.writelines([b'\x00' * width*3])
                    inp.seek(width*3,1)
                else:
                    result.writelines([inp.read(width*3)])
                result.writelines([inp.read(alignment)])
                
half_image('logo.bmp', 'halfed_logo.bmp')

Задание 1 - 3 балла     
3 балла - работает всегда       
2 балла - не работает в одном случае (нечётное количество строк/отрицательная высота/выравнивание байт)     
1 балла - не работает в двух случаях        
0 - иначе

Результат:  

<img src="halfed_logo.bmp" width="200" height="200">

----
__Из-за ошибки при сканировании изображения, это изображение получилось перевёрнутым.__         
__Тебя попросили вернуть изображению правильную ориентацию__
# Задание 1 (Варинат 2):   
Напишите функцию (метод) ```vertical_reverse_image(input_image, result_image)```
, которая изменит порядок строк изображения на противоположный.    
У этой функции должно быть два аргумета:     
- ```input_image : string``` - относительный или абсолютный путь к BMP изображению описанного выше формата.
- ```result_image : string``` - относительный или абсолютный путь, по которому будет записан результат выполнения функции.   
    
Эта функция (метод) должен использовать только функции стандартных библиотек вашего языка программирования.


In [3]:
# Возможное решение на python
def vertical_reverse_image(input_image : str, result_image: str):
    with open(input_image, 'rb') as inp:
        header = inp.read(54)
        width = int.from_bytes(header[18:22], byteorder='little')
        heigth = int.from_bytes(header[22:26], byteorder='little', signed=True)
        alignment = width * 3 % 4

        with open(result_image, 'wb') as result:
            result.writelines([header])

            inp.seek((heigth - 1)*(width*3 + alignment) + 54)
            for i in range(heigth - 1):
                result.writelines([inp.read(width*3 + alignment)])
                inp.seek(-2 * (width*3 + alignment), 1)

vertical_reverse_image('logo.bmp', 'reversed_logo.bmp')        

In [10]:
#Возможное решение 2
##TODO поменять значение высоты на такое же отрицательное

0.6666666666666666

Результат: 

<img src="reversed_logo.bmp" width="200" height="200">

----
__Дирекция очень довольна твоей работой и захотела отправить это изображение по инетрнету из Орбиты в Красноярск, но интернет в Орибте очень медленный, а наше изображение весит целых 7,5 Мегабайт!__     
__Тебя попросили сжать это изображение так, чтобы при разжатии можно было восстановить, в точности то же самое изображение.__

Давайте заметим, что наше изображение имеет не слишком много различных цветов. Каждый пиксель кодирует 2^24 возможных цветов, хотя иcпользвуется явно меньше 100. Получается, что наш файл хранит излишнюю информацию, и если избавиться от части излишней информации, то мы сможем сократить размер файла.     

# Задание 2 (Вариант 1)
1. Напишите две функции ```compress(input_image, result_image)``` и ```decompress(input_image, result_image)```, которые будут сжимать и разжимать обратно файл изображения по заданному алгоритму.
    - ```input_image : string``` - относительный или абсолютный путь к BMP изображению описанного выше формата. Гарантируется, что во входном изображении не более 256 различных цветов из 24-битной rgb палитры.
    - ```result_image : string``` - относительный или абсолютный путь, по которому будет записан результат выполнения функции.   

    Результатом сжатия (функции ```compress()```) должен стать файл формата ".cmp", который получается из оригинального BMP файла по следующему алгоритму:
    1. Первые 54 байта файла .cmp такие же как и файла .bmp
    1. Следующие 256 * 3 байта идёт "таблица использованных цветов". Это последовательность цветов, которые используются в данном изображении. 
    1. Далее идёт пиксельная информация, но каждый пиксель теперь указывается не тремя байтами rgb палитры, а одним байтом - номером соответствующего цвета слева в "таблице цветов"

    Результатом разжатия (функции ```decompress()```) должно стать BMP изображение, разжатое из файла ".cmp"         
    Эта функции (методы) должны использовать только функции стандартных библиотек вашего языка программирования.
1. Во сколько раз примерно наш метод сжимает такие изображения?

In [8]:
# Возможное решение на python
##TODO 
(128*3+1)/(128*3)

1.0026041666666667

-----
__Дирекция очень довольна твоей работой и захотела отправить это изображение по инетрнету из Орбиты в Красноярск, но интернет в Орибте очень медленный, а наше изображение весит целых 7,5 Мегабайт!__     
__Тебя попросили сжать это изображение так, чтобы при разжатии можно было восстановить, в точности то же самое изображение.__

Давайте посмотрим на верхнюю пиксельную строку нашего изображения. Она же состоит из белого пикселя "ff ff ff" повторенного много раз (столько раз, сколько влезает в одну строку изображения).     
Но!     
Мы же можем сильно сокртатить запись этого изображения, если вместо текущей записи укажем что-то вроде      
**"пиксель ff ff ff стоит 1629 раз"**, или просто **"ff ff ff 1629"**.      
Вспомним теперь, что числа у нас тоже хранятся в байтах, поэтому под число повторов мы выделим один байт. Тогда запись будет выглядеть следующим образом: **"ff ff ff 256, ff ff ff 256, ...,  ff ff ff 93"**.      
Но даже такая запись сильно сократит размер нашего файла, ведь теперь первая строка будет занимать **28 вместо 4887 байт!**  

# Задание 2 (Вариант 2)
1. Напишите две функции ```compress(input_image, result_image)``` и ```decompress(input_image, result_image)```, которые будут сжимать и разжимать обратно файл изображения по заданному алгоритму.
    - ```input_image : string``` - относительный или абсолютный путь к BMP изображению описанного выше формата. Гарантируется, что во входном изображении не более 256 различных цветов из 24-битной rgb палитры.
    - ```result_image : string``` - относительный или абсолютный путь, по которому будет записан результат выполнения функции.   

    Результатом сжатия (функции ```compress()```) должен стать файл формата ".cmp", который получается из оригинального BMP файла по следующему алгоритму:
    1. Первые 54 байта файла .cmp такие же как и файла .bmp
    1. Далее идёт пиксельная информация: каждый пиксель теперь кодируется следующим образом: сначала 3 байта цвета пикселя в палитре rgb, а потом один байт - количество раз, которое этот цвет встречается подряд

    Результатом разжатия (функции ```decompress()```) должно стать BMP изображение, разжатое из файла ".cmp"         
    Эти функции должны использовать только функции стандартных библиотек вашего языка программирования.
1. Во сколько раз такой метод сжимает изображения в лучшем случае? а в худшем?

Баллы за 2 задание:
- 3 балла за первую часть
- 2 балла за вторую

In [16]:
# Возможное решение на python
##TODO 

24

----
# Задание 3 (Вариант 2)
1. Какие недостатки вы нашли в реализованном алгоритме сжатия?     

Некоторые проблемы можно устранить если не пытаться при сжатии сохранять всю информацию, имеющуюся в изображении. Например, если нескольким пикселям, которые имели раньше разные цвета при сжатии присваивать одинаковый.
2. Напишите функцию ```super_compress(input_image, result_image)``` , которая улучшит степень сжатия файла алгоритмом из задания 2, засчёт потери части информации, и функцию ```super_decompress(input_image, result_image)```, которая восстановит изображение BMP по сохранившейся информации. Постарайтесь сделать так, чтобы изначальное и изображение в результате последовательного сжатия и разжатия не сильно отличались друг от друга.

- 1 часть 1 балл
- 2 часть 6 баллов:
    - 4 балла код
    - 2 балла пояснения